# NLP Project Group K
## Members:
### 1. Poh Soon Heng B032010010
### 2. Sue Chen Xiang B032010034
### 3. Beh Kar Soon B032010466
### 4. Kishwanth a/l Hari Krishnan B032010185

## Import the neccesary libraries

In [116]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
import random
import re
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.callbacks import TensorBoard

## Read in the training data file which name as intents

In [117]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']

with open('intents.json', "r", encoding="utf8") as f:
    data_file = f.read()
    intents = json.loads(data_file)

intents

{'intents': [{'tag': 'Greetings',
   'patterns': ['How are you',
    'Hi',
    'Hello',
    'Good day',
    "What's up",
    'Nice to meet you',
    'Yo'],
   'responses': ["Hi, I'm MeowBot! What can I help you?"]},
  {'tag': 'Unknown',
   'patterns': [''],
   'responses': ['Meow, please enter something.',
    'Please enter something so that I can help you.',
    "Meow? I can't understand you."]},
  {'tag': 'chatbot function',
   'patterns': ['What can you do for me?', 'What is your function?'],
   'responses': ['MeowBot can provide you the information about:<br>1. Cat care tips<br>2. Cat anatomy<br>3. Cat health<br>4. Cat communication cues<br>5. Quirky cat behavior<br>6. Cat facts<br>7. Cat diseases']},
  {'tag': 'Cat care tips',
   'patterns': ['Show me some cat care tips',
    'Tips to keep my cat healthy',
    'Show me more cat care tips',
    'Show me more tips',
    'Is there any other tips'],
   'responses': ["1. Groom your cat regularly<br>2. Provide fresh water daily<br>3. Ma

In [118]:
# function that remove special character and digit
def remove_special_character(sentence, remove_digits=False, keep_apostrophes=False):
    sentence = sentence.strip()

    if remove_digits and keep_apostrophes:
        pattern = r"[^a-zA-Z'\s]"
        filtered_sentence = re.sub(pattern, r'', sentence)
        return filtered_sentence
    elif keep_apostrophes:
        pattern = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(pattern, r'', sentence)
        return filtered_sentence
    elif remove_digits:
        pattern = r'[^a-zA-Z\s]'
        filtered_sentence = re.sub(pattern, r'', sentence)
        return filtered_sentence

    pattern = r'[^a-zA-Z0-9\s]'
    filtered_sentence = re.sub(pattern, "", sentence)
    return filtered_sentence

In [119]:
from contractions import CONTRACTION_MAP

# function that expand contrections
def expand_contractions(sentence, contraction_mapping=CONTRACTION_MAP):
    contraction_pattern = re.compile("({})".format('|'.join(CONTRACTION_MAP.keys())),
                                    flags=re.IGNORECASE|re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction

    try:
        expanded_sent = contraction_pattern.sub(expand_match, sentence)
        expanded_sent = re.sub("'", "", expanded_sent)
    except:
        return sentence
        
    return expanded_sent

In [120]:
from nltk.stem import WordNetLemmatizer

def lemmatization(sentence):
    lemmatizer = WordNetLemmatizer()

    tokens = nltk.word_tokenize(sentence)
    lemma_sentence = []
    for token in tokens:
        lemma = lemmatizer.lemmatize(token)
        lemma_word = token[0] + lemma[1:]
        lemma_sentence.append(lemma_word)

    lemma_sentence = " ".join(lemma_sentence)
    lemma_sentence = re.sub(r"\s\.", ".", lemma_sentence)
    return lemma_sentence

In [121]:
def preprocessing_text(text):
    text = remove_special_character(text)
    text = expand_contractions(text)
    text = lemmatization(text)

    return text

In [122]:
preprocessing_text(intents["intents"][0]["patterns"][5])
# intents["intents"][0]["patterns"]

'Nice to meet you'

## Extract unique tags and pattern from the json file
## pattern = input data
## tags = labels / classes / output data

In [123]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        # cleaning and preprocessing the text
        pattern = preprocessing_text(pattern)
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

In [124]:
documents

[(['How', 'are', 'you'], 'Greetings'),
 (['Hi'], 'Greetings'),
 (['Hello'], 'Greetings'),
 (['Good', 'day'], 'Greetings'),
 (['Whats', 'up'], 'Greetings'),
 (['Nice', 'to', 'meet', 'you'], 'Greetings'),
 (['Yo'], 'Greetings'),
 ([], 'Unknown'),
 (['What', 'can', 'you', 'do', 'for', 'me'], 'chatbot function'),
 (['What', 'is', 'your', 'function'], 'chatbot function'),
 (['Show', 'me', 'some', 'cat', 'care', 'tip'], 'Cat care tips'),
 (['Tips', 'to', 'keep', 'my', 'cat', 'healthy'], 'Cat care tips'),
 (['Show', 'me', 'more', 'cat', 'care', 'tip'], 'Cat care tips'),
 (['Show', 'me', 'more', 'tip'], 'Cat care tips'),
 (['Is', 'there', 'any', 'other', 'tip'], 'Cat care tips'),
 (['Why', 'is', 'it', 'important', 'to', 'groom', 'my', 'pet', 'regularly'],
  'Groom cat'),
 (['Why', 'should', 'I', 'groom', 'my', 'cat'], 'Groom cat'),
 (['Groom', 'cat'], 'Groom cat'),
 (['Groom', 'regularly'], 'Groom cat'),
 (['Why',
   'is',
   'it',
   'important',
   'to',
   'provide',
   'fresh',
   'water',

In [125]:
words

['a',
 'about',
 'again',
 'age',
 'am',
 'anatomy',
 'and',
 'animal',
 'any',
 'are',
 'avoid',
 'awesome',
 'behaviour',
 'best',
 'bowl',
 'box',
 'by',
 'bye',
 'can',
 'cancer',
 'car',
 'care',
 'carrier',
 'cat',
 'catfriendly',
 'cause',
 'clean',
 'common',
 'communication',
 'cue',
 'daily',
 'day',
 'designed',
 'detect',
 'diabetes',
 'diagnose',
 'disease',
 'do',
 'doesnt',
 'doing',
 'dont',
 'drink',
 'dumb',
 'enough',
 'explain',
 'fact',
 'factor',
 'feline',
 'felv',
 'fine',
 'fiv',
 'for',
 'fresh',
 'from',
 'fuck',
 'function',
 'get',
 'getting',
 'go',
 'good',
 'goodbye',
 'great',
 'groom',
 'handle',
 'happen',
 'have',
 'health',
 'healthy',
 'heartworm',
 'hello',
 'helpful',
 'helping',
 'hi',
 'highrise',
 'how',
 'hr',
 'i',
 'identify',
 'idiot',
 'if',
 'im',
 'immunodeficiency',
 'importance',
 'important',
 'in',
 'infected',
 'infection',
 'inside',
 'interacts',
 'is',
 'it',
 'joke',
 'keep',
 'laugh',
 'litter',
 'lost',
 'made',
 'make',
 'ma

In [126]:
classes

['Cancer prevention',
 'Cat care tips',
 'Cat carrier',
 'Causes of URIs',
 'Causes of cat cancer',
 "Don't drink water from bowl",
 'FIV',
 'FIV diagnose',
 'FIV prevention',
 'FIV symptoms',
 'FIV transmission',
 'FIV treatment',
 'FelV',
 'FelV diagnose',
 'FelV prevention',
 'FelV symptoms',
 'Fresh water',
 'Greetings',
 'Groom cat',
 'Heartworm diagnose',
 'Heartworm prevention',
 'Heartworm symptoms',
 'High-Rise Syndrome',
 'High-Rise Syndrome prevention',
 'Litter boxes',
 'Rabies',
 'Rabies diagnose',
 'Rabies prevention',
 'Rabies symptoms',
 'Rabies transmission',
 'Ringworm diagnose',
 'Ringworm symptoms',
 'Ringworm transmission',
 'Ringworm treatment',
 'Scratching post',
 'URIs diagnose',
 'URIs treatment',
 'URIs_prevention',
 'URIs_symptoms',
 'Unknown',
 'Upper Respiratory Infections',
 'Urinate outside litter box',
 'Worm_Treatment',
 'Worms',
 'Worms_prevention',
 'Worms_symptoms',
 'activity',
 'age',
 'appreciate',
 'cancer',
 'cancer symptoms',
 'cat anatomy',
 

In [127]:
# documents = combination between tokenized patterns and tags
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

194 documents
76 classes ['Cancer prevention', 'Cat care tips', 'Cat carrier', 'Causes of URIs', 'Causes of cat cancer', "Don't drink water from bowl", 'FIV', 'FIV diagnose', 'FIV prevention', 'FIV symptoms', 'FIV transmission', 'FIV treatment', 'FelV', 'FelV diagnose', 'FelV prevention', 'FelV symptoms', 'Fresh water', 'Greetings', 'Groom cat', 'Heartworm diagnose', 'Heartworm prevention', 'Heartworm symptoms', 'High-Rise Syndrome', 'High-Rise Syndrome prevention', 'Litter boxes', 'Rabies', 'Rabies diagnose', 'Rabies prevention', 'Rabies symptoms', 'Rabies transmission', 'Ringworm diagnose', 'Ringworm symptoms', 'Ringworm transmission', 'Ringworm treatment', 'Scratching post', 'URIs diagnose', 'URIs treatment', 'URIs_prevention', 'URIs_symptoms', 'Unknown', 'Upper Respiratory Infections', 'Urinate outside litter box', 'Worm_Treatment', 'Worms', 'Worms_prevention', 'Worms_symptoms', 'activity', 'age', 'appreciate', 'cancer', 'cancer symptoms', 'cat anatomy', 'cat communication cues', '

## Save the input data and labels into binary

In [128]:
with open("input_data.pkl", "wb") as f:
    pickle.dump(words, f)

with open("labels.pkl", "wb") as f:
    pickle.dump(classes, f)

In [129]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

## Feature Engineering: Bag of Words (BOW)

In [130]:
import pandas as pd

pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

bag = pd.DataFrame(columns=words)
# training set, bag of words for each sentence
for i, doc in enumerate(documents):
    # initialize our bag of words
    bow = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bow.append(1) if w in pattern_words else bow.append(0)

    b = {label: ba for label, ba in zip(words, bow)}
    bag = pd.concat([bag, pd.DataFrame(b, index=[i])])

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bow, output_row])
bag

,a,about,again,age,am,anatomy,and,animal,any,are,avoid,awesome,behaviour,best,bowl,box,by,bye,can,cancer,car,care,carrier,cat,catfriendly,cause,clean,common,communication,cue,daily,day,designed,detect,diabetes,diagnose,disease,do,doesnt,doing,dont,drink,dumb,enough,explain,fact,factor,feline,felv,fine,fiv,for,fresh,from,fuck,function,get,getting,go,good,goodbye,great,groom,handle,happen,have,health,healthy,heartworm,hello,helpful,helping,hi,highrise,how,hr,i,identify,idiot,if,im,immunodeficiency,importance,important,in,infected,infection,inside,interacts,is,it,joke,keep,laugh,litter,lost,made,make,mangage,me,meet,more,my,need,nice,now,of,old,other,outside,pet,physiology,please,post,prevent,prevention,programmed,provide,quirky,rabid,rabies,reason,regular,regularly,respiratory,ringworm,schedule,scratching,see,seem,should,show,shut,sign,some,stretching,stupid,sure,symptom,syndrome,teeth,tell,thank,thanks,that,thats,the,there,they,tip,to,train,transmission,transmit,transmitted,treat,type,up,upper,upto,urinate,uris,use,vet,veterinary,virus,visit,water,way,wellness,were,what,whats,when,who,why,with,worm,yo,you,your
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [131]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - tags
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

Training data created


C:\Users\User\AppData\Local\Temp\ipykernel_22400\329401055.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


## Define the neural network model

In [132]:
tensorboard = TensorBoard(log_dir="logs/chatbot_metrics")
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 128)               23296     
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dropout_15 (Dropout)        (None, 64)                0         
                                                                 
 dense_23 (Dense)            (None, 76)                4940      
                                                                 
Total params: 36,492
Trainable params: 36,492
Non-trainable params: 0
_________________________________________________________________


d:\Development\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


## Train the model

In [133]:
# fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1, callbacks=[tensorboard])
model.save("cat_disease_chatbot_model.h5", hist)

print("model created")

Epoch 1/200
39/39 [==============================] - 1s 2ms/step - loss: 4.3525 - accuracy: 0.0052
Epoch 2/200
39/39 [==============================] - 0s 2ms/step - loss: 4.3031 - accuracy: 0.0361
Epoch 3/200
39/39 [==============================] - 0s 1ms/step - loss: 4.2668 - accuracy: 0.0155
Epoch 4/200
39/39 [==============================] - 0s 1ms/step - loss: 4.1863 - accuracy: 0.0722
Epoch 5/200
39/39 [==============================] - 0s 1ms/step - loss: 4.1397 - accuracy: 0.0722
Epoch 6/200
39/39 [==============================] - 0s 2ms/step - loss: 4.0319 - accuracy: 0.0928
Epoch 7/200
39/39 [==============================] - 0s 2ms/step - loss: 3.9103 - accuracy: 0.1237
Epoch 8/200
39/39 [==============================] - 0s 2ms/step - loss: 3.8248 - accuracy: 0.0979
Epoch 9/200
39/39 [==============================] - 0s 2ms/step - loss: 3.6743 - accuracy: 0.1443
Epoch 10/200
39/39 [==============================] - 0s 2ms/step - loss: 3.6226 - accuracy: 0.1134
Epoch 11/